In [1]:
#Set up notebook
import awkward as ak
import uproot
import numpy as np

# Advanced Analysis Techniques
Sometimes you will need to perform a calculation that is not straightforward to implement. The following examples can be used as inspiration for solving trickier problems where the basic techniques are not enough. Awkward array has a wide range of functionality that may not be obvious where it is helpful.

## SUSY ISR Jet Finding Algorithm
From the SUSY run 2 stop search, there was as need to calculate whether an event contained an ISR jet subject to some quality criteria. The problem is defined as follows.

An ISR jet is the highest pT jet in an event with $P_T > 200 GeV$, $|\eta| < 2.4$, is not tagged as a b jet, it's 2 leading subjets are not tagged as b jets, and $|Jet_{\phi} - MET_{\phi}| >= 2.0$. This jet is infered to be the ISR that produces the $\tilde{t}\bar{\tilde{t}}$ pair.

First, lets compare the run-time of the columnar analysis with the original row-wise analysis. Then we will convince ourselves this gives the same result.

In [2]:
#Open and read the ROOT File TTree Events
events = uproot.open("SMS_T2tt_mStop250_mLSP75_fastsim_2016_Skim_070650_37_082253_36.root:Events")
events = events.arrays(
    expressions = ["FatJet_pt",
                   "FatJet_eta",
                   "FatJet_phi",
                   "FatJet_btagDeepB",
                   "FatJet_subJetIdx1",
                   "FatJet_subJetIdx2",
                   "SubJet_btagDeepB",
                   "MET_phi"
                  ]
)
#Helper function to calculate delta_phi between a and b
def delta_phi(a, b):
    return (a - b + np.pi) % np.pi - np.pi


In [3]:
%%time
#ISR check
FatJets_pts = ak.firsts(events["FatJet_pt"], axis=1)
FatJets_eta = ak.firsts(events["FatJet_eta"], axis=1)
FatJets_phi = ak.firsts(events["FatJet_phi"], axis=1)
FatJets_btag = ak.firsts(events["FatJet_btagDeepB"], axis=1)

#Subjet identification and tagging
Subjets_btag = events["SubJet_btagDeepB"]
nbjets = ak.num(Subjets_btag, axis=1)
FatJets_subJetIdx1 = ak.firsts(events["FatJet_subJetIdx1"], axis=1)
FatJets_subJetIdx1 = ak.mask(FatJets_subJetIdx1, FatJets_subJetIdx1 >= 0)
FatJets_subJetIdx2 = ak.firsts(events["FatJet_subJetIdx2"], axis=1)
FatJets_subJetIdx2 = ak.mask(FatJets_subJetIdx2, FatJets_subJetIdx2 >= 0)
Subjet1_btag = ak.firsts(Subjets_btag[ak.singletons(FatJets_subJetIdx1)])
Subjet2_btag = ak.firsts(Subjets_btag[ak.singletons(FatJets_subJetIdx2)])
#For events with no subjets, say there is a proxy subjet that isn't btagged
FatJets_subJetIdx1 = ak.fill_none(FatJets_subJetIdx1, 0)
FatJets_subJetIdx2 = ak.fill_none(FatJets_subJetIdx2, 0)
Subjet1_btag = ak.fill_none(Subjet1_btag, 0)
Subjet2_btag = ak.fill_none(Subjet2_btag, 0)
MET_phi = events["MET_phi"]
working_point = 0.2217
SAT_Pass_ISR_columnar = (
        (FatJets_pts >= 200.)
        & (abs(FatJets_eta) < 2.4)
        & (FatJets_btag < working_point)
        #subJet1
        & (~((FatJets_subJetIdx1 >= 0) & (FatJets_subJetIdx1 < nbjets) & (Subjet1_btag > working_point)))
        #subJet2
        & (~((FatJets_subJetIdx2 >= 0) & (FatJets_subJetIdx2 < nbjets) & (Subjet2_btag > working_point)))
        & (abs(delta_phi(FatJets_phi, MET_phi)) >= 2.0)
        )
SAT_Pass_ISR_columnar = ak.fill_none(SAT_Pass_ISR_columnar, False)

CPU times: user 436 ms, sys: 23.2 ms, total: 459 ms
Wall time: 457 ms


In [4]:
%%time 
#this can take a couple minutes
#ISR check
SAT_Pass_ISR_row = (np.zeros(len(events))==1)
FatJets_pts = events["FatJet_pt"]
FatJets_eta = events["FatJet_eta"]
FatJets_phi = events["FatJet_phi"]
FatJets_btag = events["FatJet_btagDeepB"]
FatJets_subJetIdx1 = events["FatJet_subJetIdx1"]
FatJets_subJetIdx2 = events["FatJet_subJetIdx2"]
SubJets_btag = events["SubJet_btagDeepB"]
MET_phi = events["MET_phi"]
working_point = 0.2217 #2016 taken from SUSYANATools constants
for i, pts in enumerate(FatJets_pts):
    if len(pts) == 0:
        continue
    if (np.float32(pts[0]) < 200.0): #Only use leading fatjet
        continue
    if (np.abs(FatJets_eta[i][0]) > 2.4):
        continue
    if (FatJets_btag[i][0] > working_point):
        continue
    nSubJets = len(SubJets_btag[i])
    if (FatJets_subJetIdx1[i][0] >= 0 and FatJets_subJetIdx1[i][0] < nSubJets and SubJets_btag[i][FatJets_subJetIdx1[i][0]] > working_point):
        continue
    if (FatJets_subJetIdx2[i][0] >= 0 and FatJets_subJetIdx2[i][0] < nSubJets and SubJets_btag[i][FatJets_subJetIdx2[i][0]] > working_point):
        continue
    dphi = np.abs(delta_phi(FatJets_phi[i][0], MET_phi[i]))
    if dphi >= 2.0:
        SAT_Pass_ISR_row[i] = True

CPU times: user 1min 41s, sys: 3.38 ms, total: 1min 41s
Wall time: 1min 41s


In [5]:
if ak.all(SAT_Pass_ISR_columnar == SAT_Pass_ISR_row):
    print("Methods produce same result")

Methods produce same result


Obviously the row-wise analysis is easier to implement, but the run-time is about 200 times longer. With large sets of data a script that original took 1 day to finish could instead have been finished within ~7 minutes. This not only allows your analysis to proceed quicker, but also allows other users to get access to the cluster much sooner and reduces the carbon footprint of our analyses. 

Let's move on to understanding the different parts of the columnar analysis code.
### Columnar Script Analysis

In [6]:
FatJets_pts = ak.firsts(events["FatJet_pt"], axis=1)
FatJets_eta = ak.firsts(events["FatJet_eta"], axis=1)
FatJets_phi = ak.firsts(events["FatJet_phi"], axis=1)
FatJets_btag = ak.firsts(events["FatJet_btagDeepB"], axis=1)

In this step we are obtaining the variables relevant to the leading FatJet in each event. `ak.firsts()` "selects the first element of each non-empty list and inserts None for each empty list." Therefore, events without a leading FatJet will read `None`.

In [7]:
#Subjet identification and tagging
Subjets_btag = events["SubJet_btagDeepB"]
nbjets = ak.num(Subjets_btag, axis=1)
FatJets_subJetIdx1 = ak.firsts(events["FatJet_subJetIdx1"], axis=1)
FatJets_subJetIdx1 = ak.mask(FatJets_subJetIdx1, FatJets_subJetIdx1 >= 0)
FatJets_subJetIdx2 = ak.firsts(events["FatJet_subJetIdx2"], axis=1)
FatJets_subJetIdx2 = ak.mask(FatJets_subJetIdx2, FatJets_subJetIdx2 >= 0)
Subjet1_btag = ak.firsts(Subjets_btag[ak.singletons(FatJets_subJetIdx1)])
Subjet2_btag = ak.firsts(Subjets_btag[ak.singletons(FatJets_subJetIdx2)])
#For events with no subjets, say there is a proxy subjet that isn't btagged
FatJets_subJetIdx1 = ak.fill_none(FatJets_subJetIdx1, 0)
FatJets_subJetIdx2 = ak.fill_none(FatJets_subJetIdx2, 0)
Subjet1_btag = ak.fill_none(Subjet1_btag, 0)
Subjet2_btag = ak.fill_none(Subjet2_btag, 0)

The tricky part for this algorithm is handling the subjets of the fat jet. In any event, the leading fat jet may have 0, 1, or 2 subjets, or if there are no fatjets there will be no subjets. In the MC file, the absence of a certain subjet is given by an index of -1. 

In [8]:
FatJets_subJetIdx1 = ak.firsts(events["FatJet_subJetIdx1"], axis=1)
FatJets_subJetIdx1 = ak.mask(FatJets_subJetIdx1, FatJets_subJetIdx1 >= 0)
Subjet1_btag = ak.firsts(Subjets_btag[ak.singletons(FatJets_subJetIdx1)])

We call `ak.firsts(events["FatJet_subJetIdx1"], axis=1)` to get the leading fat jet's subjet 1 index. We then call `ak.mask(FatJets_subJetIdx1, FatJets_subJetIdx1 >= 0)` to replace any index < 0 with a None value indicating the absence of a subjet. 

`Subjets_btag` contains the btag values for all fatjets in the event. We want to mask `Subjets_btag` in such a way to obtain the btag value for subjet 1 and 2 in two separate arrays. However `FatJets_subJetIdx1` is of depth 1 and `Subjets_btag` is of depth 2. We must mask `Subjets_btag` with an array of depth 2 in order to obtain an array of depth 1. `ak.singletons(FatJets_subJetIdx1)` essentially increases the depth of the argument by 1. See the example below.

In [9]:
ak.singletons([1,2,3])

<Array [[1], [2], [3]] type='3 * var * int64'>

In [10]:
#For events with no subjets, say there is a proxy subjet that isn't btagged
FatJets_subJetIdx1 = ak.fill_none(FatJets_subJetIdx1, 0)
FatJets_subJetIdx2 = ak.fill_none(FatJets_subJetIdx2, 0)
Subjet1_btag = ak.fill_none(Subjet1_btag, 0)
Subjet2_btag = ak.fill_none(Subjet2_btag, 0)

Now we must handle the events missing subjets. `ak.fill_none()` replaces all `None` entries in an array with a given value. Any time there is no subjet, we will say the subjet index is 0 and it has a btag value of 0.

In [11]:
SAT_Pass_ISR_columnar = (
        (FatJets_pts >= 200.)
        & (abs(FatJets_eta) < 2.4)
        & (FatJets_btag < working_point)
        #subJet1
        & (~((FatJets_subJetIdx1 >= 0) & (FatJets_subJetIdx1 < nbjets) & (Subjet1_btag > working_point)))
        #subJet2
        & (~((FatJets_subJetIdx2 >= 0) & (FatJets_subJetIdx2 < nbjets) & (Subjet2_btag > working_point)))
        & (abs(delta_phi(FatJets_phi, MET_phi)) >= 2.0)
        )
SAT_Pass_ISR_columnar = ak.fill_none(SAT_Pass_ISR_columnar, False)
SAT_Pass_ISR_columnar

<Array [False, False, True, ..., False, False, False] type='847873 * bool'>

In this step we finally apply all the selection criteria and obtain `SAT_Pass_ISR_columnar` which is an array of booleans defining which events contain an ISR jet.